###  Retrieval-Augmented Generation (RAG) for Inventory Scoring

#### Import Packages

In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join('..')))

In [2]:
from rag.rag_pipeline import Rag
import pandas as pd

/home/babi/miniconda3/envs/week10/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
2024-03-23 16:37:05,232 - DEBUG - load_ssl_context verify=True cert=None trust_env=True http2=False
2024-03-23 16:37:05,236 - DEBUG - load_verify_locations cafile='/home/babi/miniconda3/envs/week10/lib/python3.8/site-packages/certifi/cacert.pem'
2024-03-23 16:37:05,247 - DEBUG - load_ssl_context verify=True cert=None trust_env=True http2=False
2024-03-23 16:37:05,248 - DEBUG - load_verify_locations cafile='/home/babi/miniconda3/envs/week10/lib/python3.8/site-packages/certifi/cacert.pem'


In [3]:
rag_chain = Rag()

/home/babi/miniconda3/envs/week10/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-23 16:37:09,878 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2024-03-23 16:37:09,882 - DEBUG - Starting new HTTPS connection (1): huggingface.co:443
2024-03-23 16:37:10,477 - DEBUG - https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json HTTP/1.1" 200 0
2024-03-23 16:37:10,698 - DEBUG - https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/config_sentence_transformers.json HTTP/1.1" 200 0
2024-03-23 16:37:10,903 - DEBUG - https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/README.md HTTP/1.1" 200 0
2024-03-23 16:37:11,115 - DEBUG - https://huggingface.co:443 "HEAD /sentence-transformers/all

#### Populate Merged Data into vector database

In [4]:
merged_df = pd.read_csv("../data/merged_data.csv")

2024-03-23 16:37:13,431 - DEBUG - Starting new HTTPS connection (1): us-api.i.posthog.com:443
2024-03-23 16:37:15,178 - DEBUG - https://us-api.i.posthog.com:443 "POST /batch/ HTTP/1.1" 200 None


In [5]:
merged_df.head()

,Unnamed: 0,campaign_id,budget_amount,budget_currencycode,KPI,pricing_model,geo_targeting,vertical,targeting,game_key,...,devicetype,devicemake,site_name,matchedfoldposition,browser,impression,engagement,click,CTR,ER
0,0,f4p5a01,277.12,USD,Engagement,CPE,All Country,Sports,Opinion Leaders| Investors| Policy Makers| C&I...,b4f31c796c130fd594d253ba9336a01d/7ae9931edb2ed...,...,Tablet,Samsung,www.yahoo.com,Unknown,Chrome,57.0,0.0,0.0,0.000000,0.000000
1,1,f4p5a01,277.12,USD,Engagement,CPE,All Country,Sports,Opinion Leaders| Investors| Policy Makers| C&I...,b4f31c796c130fd594d253ba9336a01d/7ae9931edb2ed...,...,Mobile,Apple,www.yahoo.com,Unknown,Mobile Safari,983.0,265.0,71.0,7.222787,26.958291
2,2,f4p5a01,277.12,USD,Engagement,CPE,All Country,Sports,Opinion Leaders| Investors| Policy Makers| C&I...,b4f31c796c130fd594d253ba9336a01d/7ae9931edb2ed...,...,Mobile,Samsung,www.yahoo.com,Unknown,Chrome,9.0,0.0,0.0,0.000000,0.000000
3,3,f4p5a01,277.12,USD,Engagement,CPE,All Country,Sports,Opinion Leaders| Investors| Policy Makers| C&I...,b4f31c796c130fd594d253ba9336a01d/7ae9931edb2ed...,...,Mobile,Apple,www.the-sun.com,Unknown,Mobile Safari UI/WKWebView,3.0,0.0,0.0,0.000000,0.000000
4,4,f4p5a01,277.12,USD,Engagement,CPE,All Country,Sports,Opinion Leaders| Investors| Policy Makers| C&I...,b4f31c796c130fd594d253ba9336a01d/7ae9931edb2ed...,...,Mobile,Samsung,currently.att.yahoo.com,Unknown,Chrome Mobile,27.0,3.0,0.0,0.000000,11.111111


#### Drop unncessary columns

In [6]:
merged_df = merged_df.drop(columns=['Unnamed: 0', 'impression', 'engagement', 'click', 'budget_currencycode', 'budget_amount', 'pricing_model', 'devicemake', 'osfamily', 'devicetype'])

In [7]:
merged_df.columns

Index(['campaign_id', 'KPI', 'geo_targeting', 'vertical', 'targeting',
       'game_key', 'adformat', 'creative', 'renderingcontext', 'site_name',
       'matchedfoldposition', 'browser', 'CTR', 'ER'],
      dtype='object')

#### Sampling data

In [8]:
data_group_by_campaign = merged_df['campaign_id'].value_counts()
data_group_by_campaign

campaign_id
p19pznm    497402
lj8gesi    404475
nzup6yr    302474
noaiquo     83504
woln7nu     78826
            ...  
6rlgajs      2283
vsi8o9t      1734
np5elle      1673
quhw0xz      1220
cvpj79c       806
Name: count, Length: 78, dtype: int64

In [9]:
filtered_df = merged_df[(merged_df['geo_targeting'] != 'All Country') & (merged_df['CTR'] != 0) & (merged_df['ER'] != 0)]

def sample_rows(group):
    return group.sample(n=min(len(group), 100), random_state=1)

sampled_df = filtered_df.groupby('campaign_id').apply(sample_rows).reset_index(drop=True)

In [10]:
sampled_df.shape

(1100, 14)

#### Check Null values

In [11]:
# Calculate the percentage of missing values in each column
missing_percentage = sampled_df.isnull().sum() / len(merged_df) * 100

# Print the percentage of missing values for each column
print(missing_percentage)

campaign_id            0.0
KPI                    0.0
geo_targeting          0.0
vertical               0.0
targeting              0.0
game_key               0.0
adformat               0.0
creative               0.0
renderingcontext       0.0
site_name              0.0
matchedfoldposition    0.0
browser                0.0
CTR                    0.0
ER                     0.0
dtype: float64


#### Generating Embeddings

In [44]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

2024-03-23 16:53:19,347 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2024-03-23 16:53:19,354 - DEBUG - Resetting dropped connection: huggingface.co
2024-03-23 16:53:20,349 - DEBUG - https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json HTTP/1.1" 200 0
2024-03-23 16:53:20,558 - DEBUG - https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/config_sentence_transformers.json HTTP/1.1" 200 0
2024-03-23 16:53:20,785 - DEBUG - https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/README.md HTTP/1.1" 200 0
2024-03-23 16:53:21,163 - DEBUG - https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json HTTP/1.1" 200 0
2024-03-23 16:53:21,383 - DEBUG - https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/sentence_bert_config.json HTTP/1.1" 200 0
2024-03-23 16:53:21,669 - DEBUG - https://huggingface.co:4

In [ ]:
# Example: Generating embeddings for the 'vertical' column in campaign_data
sampled_df['geo_targeting'].apply(lambda x: model.encode(x))

In [19]:
from sentence_transformers import SentenceTransformer
import pandas as pd

# Initialize the model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Assuming sampled_df is your original DataFrame
# And focusing on columns that could benefit from embeddings for this example
textual_columns = ['KPI', 'geo_targeting', 'vertical', 'targeting', 'adformat', 'creative', 'renderingcontext', 'matchedfoldposition', 'site_name', 'browser']

# Initialize an empty DataFrame to store embeddings
embeddings_df = pd.DataFrame()

# Generate embeddings for each specified column and add to the new DataFrame
for column in sampled_df.columns:
    # Apply the model to each item in the column and store the result in the new DataFrame
    # Note: This creates a list of embeddings for each row, which are high-dimensional vectors
    if column in textual_columns:
        embeddings_df[f'{column}_embeddings'] = sampled_df[column].apply(lambda x: model.encode(x).tolist())
    else:
        embeddings_df[column] = sampled_df[column]


# Demonstrating the structure of the new DataFrame with embeddings
print(embeddings_df.head())
embeddings_df.to_csv("../data/sampled_embedded_data.csv")

2024-03-23 16:11:47,693 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2024-03-23 16:11:48,176 - DEBUG - https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json HTTP/1.1" 200 0
2024-03-23 16:11:48,370 - DEBUG - https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/config_sentence_transformers.json HTTP/1.1" 200 0
2024-03-23 16:11:48,559 - DEBUG - https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/README.md HTTP/1.1" 200 0
2024-03-23 16:11:48,809 - DEBUG - https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json HTTP/1.1" 200 0
2024-03-23 16:11:49,118 - DEBUG - https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/sentence_bert_config.json HTTP/1.1" 200 0
2024-03-23 16:11:49,441 - DEBUG - https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/config.json HTTP/

  campaign_id_embeddings                                     KPI_embeddings  \
0                66bpf2h  [0.024438880383968353, 0.07737276703119278, 0....   
1                66bpf2h  [0.024438880383968353, 0.07737276703119278, 0....   
2                66bpf2h  [0.024438880383968353, 0.07737276703119278, 0....   
3                66bpf2h  [0.024438880383968353, 0.07737276703119278, 0....   
4                66bpf2h  [0.024438880383968353, 0.07737276703119278, 0....   

                            geo_targeting_embeddings  \
0  [-0.009008824825286865, 0.013777172192931175, ...   
1  [-0.009008824825286865, 0.013777172192931175, ...   
2  [-0.009008824825286865, 0.013777172192931175, ...   
3  [-0.009008824825286865, 0.013777172192931175, ...   
4  [-0.009008824825286865, 0.013777172192931175, ...   

                                 vertical_embeddings  \
0  [0.036567866802215576, 0.006975037977099419, -...   
1  [0.036567866802215576, 0.006975037977099419, -...   
2  [0.03656786680221

#### Load the Emedded sample dataset

In [18]:
embedded_data = pd.read_csv("../data/sampled_embedded_data.csv")

In [20]:
embedded_data.columns

Index(['campaign_id', 'KPI_embeddings', 'geo_targeting_embeddings',
       'vertical_embeddings', 'targeting_embeddings', 'game_key',
       'adformat_embeddings', 'creative_embeddings',
       'renderingcontext_embeddings', 'site_name_embeddings',
       'matchedfoldposition_embeddings', 'browser_embeddings', 'CTR', 'ER'],
      dtype='object')

#### Create sampled Data dictionary

In [109]:
sampled_dic = embedded_data.iloc[:, :].to_dict(orient='records')

#### Create Weivate Connection

In [110]:
import weaviate
from weaviate.embedded import EmbeddedOptions


client = weaviate.Client( embedded_options = EmbeddedOptions())

/home/babi/miniconda3/envs/week10/lib/python3.8/site-packages/weaviate/warnings.py:158: DeprecationWarning: Dep016: You are using the Weaviate v3 client, which is deprecated.
            Consider upgrading to the new and improved v4 client instead!
            See here for usage: https://weaviate.io/developers/weaviate/client-libraries/python
            
  warnings.warn(
2024-03-23 17:14:02,257 - DEBUG - Starting new HTTP connection (1): localhost:8079
2024-03-23 17:14:02,259 - DEBUG - http://localhost:8079 "GET /v1/.well-known/openid-configuration HTTP/1.1" 404 0
2024-03-23 17:14:02,264 - DEBUG - Starting new HTTP connection (1): localhost:8079
2024-03-23 17:14:02,267 - DEBUG - http://localhost:8079 "GET /v1/meta HTTP/1.1" 200 871
2024-03-23 17:14:02,271 - DEBUG - Starting new HTTPS connection (1): pypi.org:443


embedded weaviate is already listening on port 8079


2024-03-23 17:14:02,528 - DEBUG - https://pypi.org:443 "GET /pypi/weaviate-client/json HTTP/1.1" 200 49836


In [111]:
# client.schema.delete_class("inventory_data")

2024-03-23 17:14:04,259 - DEBUG - http://localhost:8079 "DELETE /v1/schema/Inventory_data HTTP/1.1" 200 0


#### Create Weivate Database Schema

In [112]:
client.schema.create("../db/schema.json")

{"level":"info","msg":"Created shard inventory_data_3BRtW7h3KFC4 in 1.807381ms","time":"2024-03-23T17:14:06+03:00"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-03-23T17:14:06+03:00","took":74904}
2024-03-23 17:14:06,891 - DEBUG - http://localhost:8079 "POST /v1/schema HTTP/1.1" 200 None


#### Storing Data in a Vector Database (Weaviate)

In [113]:
for value in sampled_dic:
    client.data_object.create(value, "Inventory_data")

2024-03-23 17:14:08,621 - DEBUG - http://localhost:8079 "POST /v1/objects HTTP/1.1" 200 None
2024-03-23 17:14:08,653 - DEBUG - http://localhost:8079 "POST /v1/objects HTTP/1.1" 200 None


2024-03-23 17:14:08,680 - DEBUG - http://localhost:8079 "POST /v1/objects HTTP/1.1" 200 None
2024-03-23 17:14:08,702 - DEBUG - http://localhost:8079 "POST /v1/objects HTTP/1.1" 200 None
2024-03-23 17:14:08,733 - DEBUG - http://localhost:8079 "POST /v1/objects HTTP/1.1" 200 None
2024-03-23 17:14:08,776 - DEBUG - http://localhost:8079 "POST /v1/objects HTTP/1.1" 200 None
2024-03-23 17:14:08,807 - DEBUG - http://localhost:8079 "POST /v1/objects HTTP/1.1" 200 None
2024-03-23 17:14:08,836 - DEBUG - http://localhost:8079 "POST /v1/objects HTTP/1.1" 200 None
2024-03-23 17:14:08,862 - DEBUG - http://localhost:8079 "POST /v1/objects HTTP/1.1" 200 None
2024-03-23 17:14:08,888 - DEBUG - http://localhost:8079 "POST /v1/objects HTTP/1.1" 200 None
2024-03-23 17:14:08,916 - DEBUG - http://localhost:8079 "POST /v1/objects HTTP/1.1" 200 None
2024-03-23 17:14:08,942 - DEBUG - http://localhost:8079 "POST /v1/objects HTTP/1.1" 200 None
2024-03-23 17:14:08,967 - DEBUG - http://localhost:8079 "POST /v1/obje

In [114]:
inventory_data = client.data_object.get(class_name="Inventory_data")

2024-03-23 17:15:11,562 - DEBUG - Resetting dropped connection: localhost
2024-03-23 17:15:11,588 - DEBUG - http://localhost:8079 "GET /v1/objects?class=Inventory_data HTTP/1.1" 200 None


In [115]:
inventory_data

{'deprecations': [],
 'objects': [{'class': 'Inventory_data',
   'creationTimeUnix': 1711203258378,
   'id': '00a861d0-35cf-49a5-b00f-928e7e267813',
   'lastUpdateTimeUnix': 1711203258378,
   'properties': {'adformat_embeddings': '[0.011631528846919537, -0.00014374492457136512, -0.08005231618881226, -0.018284045159816742, -0.038870301097631454, -0.07851551473140717, 0.034235771745443344, 0.03266960754990578, -0.052845507860183716, 0.01355928648263216, 0.0020959905814379454, -0.021402720361948013, 0.05733857676386833, 0.00712199741974473, -0.07623642683029175, 0.03341192379593849, -0.05603780224919319, -0.004770269151777029, -0.05289943888783455, -0.005333209875971079, 0.004007149022072554, -0.036061134189367294, -0.059680260717868805, -0.019549449905753136, -0.0358341783285141, 0.04357834905385971, -0.027481192722916603, 0.08479361236095428, -0.005810165777802467, -0.07449507713317871, 0.05503076687455177, 0.055987849831581116, 0.004854681435972452, -0.03210281953215599, 0.000663445855

#### Retrieving Relevant Inventories


In [128]:
query_result = client.query.get("Inventory_data", ["campaign_id", "matchedfoldposition_embeddings"]).with_near_vector({
    "vector": model.encode("matchedfoldposition_embeddings Below")
}).with_limit(100).do()

print(query_result)

Batches: 100%|██████████| 1/1 [00:00<00:00, 69.09it/s]
2024-03-23 17:16:01,920 - DEBUG - http://localhost:8079 "POST /v1/graphql HTTP/1.1" 200 39


{'data': {'Get': {'Inventory_data': []}}}


{"action":"read_disk_use","level":"warning","msg":"disk usage currently at 89.17%, threshold set to 80.00%","path":"/home/babi/.local/share/weaviate","time":"2024-03-23T17:52:32+03:00"}


In [108]:
import json

# response = (
#     client.query
#     .get("inventory_data", ["campaign_id"])
#     .with_near_text({"concepts": ["sport"]})
#     .with_limit(10)
#     .do()
# )

# print(json.dumps(response, indent=4))

response = (
    client.query
    .get("inventory_data", ["campaign_id", "matchedfoldposition"])
    .with_near_text({
        "concepts": ["matchedfoldposition below"]
    })
    .with_limit(2)
    .with_additional(["distance"])
    .do()
)

print(json.dumps(response, indent=2))



2024-03-23 17:13:44,142 - DEBUG - Resetting dropped connection: localhost
{"action":"requests_total","api":"graphql","class_name":"","error":"Unknown argument \"nearText\" on field \"Inventory_data\" of type \"GetObjectsObj\". Did you mean \"nearVector\" or \"nearObject\"?","level":"error","msg":"unexpected error","query_type":"","time":"2024-03-23T17:13:44+03:00"}
2024-03-23 17:13:44,145 - DEBUG - http://localhost:8079 "POST /v1/graphql HTTP/1.1" 200 208


{
  "errors": [
    {
      "locations": [
        {
          "column": 30,
          "line": 1
        }
      ],
      "message": "Unknown argument \"nearText\" on field \"Inventory_data\" of type \"GetObjectsObj\". Did you mean \"nearVector\" or \"nearObject\"?",
      "path": null
    }
  ]
}


In [17]:
client.schema.delete_class("Inventory_data")

2024-03-22 12:59:20,647 - DEBUG - http://localhost:8079 "DELETE /v1/schema/Inventory_data HTTP/1.1" 200 0


#### Semantic search

In [14]:
response = (
    client.query
    .get("Inventory_data", ["campaign_id", "vertical", "targeting"])
    .with_near_text({"concepts": ["travel"]})
    .with_limit(10)
    .do()
)

2024-03-22 12:49:21,665 - DEBUG - Resetting dropped connection: localhost
2024-03-22 12:49:22,165 - DEBUG - http://localhost:8079 "POST /v1/graphql HTTP/1.1" 200 1288


In [15]:
response

{'data': {'Get': {'Inventory_data': [{'campaign_id': 'f4p5a01',
     'targeting': 'Opinion Leaders| Investors| Policy Makers| C&I| Golf Enthusiasts',
     'vertical': 'Sports'},
    {'campaign_id': 'f4p5a01',
     'targeting': 'Opinion Leaders| Investors| Policy Makers| C&I| Golf Enthusiasts',
     'vertical': 'Sports'},
    {'campaign_id': 'f4p5a01',
     'targeting': 'Opinion Leaders| Investors| Policy Makers| C&I| Golf Enthusiasts',
     'vertical': 'Sports'},
    {'campaign_id': 'f4p5a01',
     'targeting': 'Opinion Leaders| Investors| Policy Makers| C&I| Golf Enthusiasts',
     'vertical': 'Sports'},
    {'campaign_id': 'f4p5a01',
     'targeting': 'Opinion Leaders| Investors| Policy Makers| C&I| Golf Enthusiasts',
     'vertical': 'Sports'},
    {'campaign_id': 'f4p5a01',
     'targeting': 'Opinion Leaders| Investors| Policy Makers| C&I| Golf Enthusiasts',
     'vertical': 'Sports'},
    {'campaign_id': 'f4p5a01',
     'targeting': 'Opinion Leaders| Investors| Policy Makers| C&I|